# Multilevel Quantum Circuit Partitioning

This notebook explores the multilevel framework in some more depth and briefly comparese the different coarsening routines. This follows on from the "walkthrough" notebook.

In [ ]:
from disqco.circuits import cp_fraction
from disqco import QuantumCircuitHyperGraph
from qiskit import transpile
from disqco import set_initial_partition_assignment, calculate_full_cost
import numpy as np
import time
from disqco import QuantumNetwork

num_qubits = 16
num_partitions = 4
qpu_size = int(num_qubits / num_partitions) + 1
qpu_sizes = [qpu_size] * num_partitions

# qpu_sizes[-1] += 1

network = QuantumNetwork(qpu_sizes)


circuit = cp_fraction(  num_qubits=num_qubits,
                        depth=2*num_qubits,
                        fraction= 0.5,
                        seed=42)

# circuit = QuantumVolume(num_qubits, depth=num_qubits)

circuit = transpile(circuit, basis_gates = ['cp', 'u'])
depth = circuit.depth()

graph = QuantumCircuitHyperGraph(circuit)
assignment = set_initial_partition_assignment(graph, network)

initial_cost = calculate_full_cost(graph, assignment, num_partitions)


In [ ]:

graph.draw(network=network, show_labels=False, assignment=assignment, output="mpl")

We first run the normal FM algorithm, with no coarsening, to set a benchmark. We will use the depth of the circuit to calculate the number of passes we will use, for fairness of comparison. We will set a limit on the number of nodes that can be moved per pass.

In [ ]:
from disqco.parti import FiducciaMattheyses

level_limit = int(np.ceil(np.log2(depth)))

num_passes = (level_limit+1) * 10
move_limit_per_pass = len(graph.nodes)*0.125

start = time.time()
partitioner = FiducciaMattheyses(circuit, network, assignment, hypergraph =graph)
results = partitioner.partition(
    limit=move_limit_per_pass,
    passes=num_passes

)

end = time.time()
print("Final cost: ", results['best_cost'])
print("Time taken: ", end-start)



In [ ]:
from disqco.parti.FM.multilevel_FM import *
from disqco.graphs.coarsening.coarsener import HypergraphCoarsener

coarsener = HypergraphCoarsener()

initial_graph_full = graph.copy()

graph_list, mapping_list = coarsener.coarsen_full(initial_graph_full, num_levels = depth)

coarsest_graph = graph_list[-1]

print(len(coarsest_graph.nodes))

We start by coarsening the graph down to a single time-step, to see how we well can do with static partitioning. The initialisation and termination nodes are included in the figure but are not part of the actual graph.

In [ ]:
coarsest_graph.draw(network=network, assignment=assignment, output="mpl")

In [ ]:
partitioner = FiducciaMattheyses(circuit, network, assignment, hypergraph=coarsest_graph)

start = time.time()
results = partitioner.partition(limit=move_limit_per_pass, max_gain=4*depth, passes=num_passes, log=False)
end = time.time()

print("Final cost: ", results['best_cost'])
cost_list = results['cost_list']

print("Time taken for coarsest graph: ", end-start)


Static partitioning will be much faster, and, in some cases, static partitioning can outperform the fine-grained approach, since the problem size is smaller, but it will often be limited since no state teleportation is possible. We will now look at a multilevel approach using a window-based coarsening routine.

In [ ]:

print(len(graph.nodes))

full_coarsener = coarsener.coarsen_full
graph_list_window, mapping_list_window = full_coarsener(graph, num_levels = level_limit)

partitioner = FiducciaMattheyses(circuit, network, assignment, hypergraph=graph)
start = time.time()
results = partitioner.multilevel_partition(coarsener=full_coarsener, num_levels = level_limit, passes_per_level = int(move_limit_per_pass))
end = time.time()

assignment_list_window = results['assignment_list']

best_cost = results['best_cost']
total_time = end - start
cost_list_window = results['cost_list']

print(f'Best cost: {best_cost}')
print(f'Total time: {total_time}')

i = 0
print("Drawing figures")
for g in reversed(graph_list_window):
    fig = g.draw(network, assignment_list_window[i], qpu_sizes, show_labels=True, output='mpl')
    display(fig)
    print(f'Cost at level {i}, {cost_list_window[i]}')
    i += 1

In [ ]:
initial_graph_blocks = graph.copy()

block_coarsener = HypergraphCoarsener().coarsen_blocks
graph_list_blocks, mapping_list_blocks = block_coarsener(initial_graph_blocks, num_blocks= None, block_size = level_limit)

partitioner = FiducciaMattheyses(circuit, network, assignment, hypergraph=graph)

start = time.time()
results = partitioner.multilevel_partition(coarsener=block_coarsener, num_blocks= None, block_size = level_limit, passes_per_level = int(move_limit_per_pass))
end = time.time()

assignment_list_blocks = results['assignment_list']
best_cost = results['best_cost']
cost_list_blocks = results['cost_list']
total_time = end - start

i = 0

print(f'Best cost: {best_cost}')
print(f'Total time: {total_time}')

print("Drawing figures")

for g in reversed(graph_list_blocks):
    fig = g.draw(network, assignment_list_blocks[i], qpu_sizes, show_labels=True, output='mpl')
    display(fig)
    print(f'Cost at level {i}, {cost_list_blocks[i]}')
    i += 1

In [ ]:
from disqco.drawing.tikz_drawing import hypergraph_to_tikz
recursive_coarsener = HypergraphCoarsener().coarsen_recursive
graph_list_recursive, mapping_list_recursive = recursive_coarsener(graph)

partitioner = FiducciaMattheyses(circuit, network, assignment, hypergraph=graph)

start = time.time()
results = partitioner.multilevel_partition(coarsener=recursive_coarsener, passes_per_level = int(move_limit_per_pass))
end = time.time()

assignment_list_recursive = results['assignment_list']
cost_list_recursive = results['cost_list']  
i = 0

best_cost = results['best_cost']
total_time = end - start

print(f'Best cost: {best_cost}')
print(f'Total time: {total_time}')
print("Drawing figures")

tikz_list = []

for g in reversed(graph_list_recursive):
    fig = g.draw(network, assignment_list_recursive[i], qpu_sizes, show_labels=True, output='mpl')

    display(fig)
    print(f'Cost at level {i}, {cost_list_recursive[i]}')
    i += 1


In [ ]:
for graph_tikz in tikz_list:
    print(graph_tikz)

In [ ]:
import matplotlib.pyplot as plt

index_list = [i*10 for i in range(level_limit+1)]

fig, ax = plt.subplots(1, 1, figsize=(10, 5))
cost_list_FM = [cost_list[i*10]for i in range(level_limit+1)]

plt.plot(cost_list_FM, label = 'FM')
plt.plot(cost_list_window, label = 'Window')
plt.plot(cost_list_blocks, label = 'Blocks')
plt.plot(cost_list_recursive, label = 'Recursive')

plt.legend()
plt.xlabel('Level')
plt.ylabel('Cost')